In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/sample-sales-data/sales_data_sample.csv', encoding = 'unicode_escape')

In [ ]:
df.shape
#2823 rows and 25 columns

In [ ]:
df.head(3)

In [ ]:
df = pd.DataFrame(df)
df.head(3)

In [ ]:
#Removing the variables which dont add significant value fot the analysis.
to_drop = ['PHONE','ADDRESSLINE1','ADDRESSLINE2','STATE','POSTALCODE']
df = df.drop(to_drop, axis=1)
df

In [ ]:
cols_needed = ['CUSTOMERNAME','ORDERNUMBER','ORDERDATE','SALES']
df = df[cols_needed]
df

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes
#order_date is not in dateformat

In [ ]:
df['CUSTOMERNAME'].value_counts()

REM analysis

In [ ]:
#most recent date by customer_name
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'])
recent_date = df.groupby('CUSTOMERNAME')['ORDERDATE'].max()
recent_date

In [ ]:
# the most recent date in the entire dataset
most_recent = df['ORDERDATE'].max()
most_recent

In [ ]:
def subtract_date(dt):
    difference = (most_recent - dt).days
    return difference
recency = recent_date.apply(subtract_date)
recency

In [ ]:
frequency = df.groupby(['CUSTOMERNAME','ORDERNUMBER']).size()
frequency = frequency.groupby('CUSTOMERNAME').size()
frequency.sort_values(ascending = False)

In [ ]:
monetary = df.groupby('CUSTOMERNAME')['SALES'].sum()
monetary

Make them a table

In [ ]:
rfm_table = pd.DataFrame()
#create columns for each df series
rfm_table['recency'] = recency
rfm_table['frequency'] = frequency
rfm_table['monetary'] = monetary

In [ ]:
rfm_table

For instance, if the quartiles of recency are (80,125,230)(80,125,230), then a customer with a recency value of 7070 can be given a class value of 4 while a value of 120120 can be assigned a class value of 3 and so on. Let’s implement this below.

In [ ]:
quantile_df = rfm_table.quantile(q=[0.25,0.50,0.75])
quantile_df

This change in the order of assigning class values was made because low recency values mark the more recent customers and we want to assign them higher class values. But for frequency and monetary, higher values indicate top customers.

In [ ]:
def quantile_classes(x,quantile_values,attribute):
    #recency
    if attribute == 'recency':
        if x <= quantile_values.loc[0.25,attribute]:
            return '4'
        elif x >= quantile_values.loc[0.25,attribute] and x<=quantile_values.loc[0.50,attribute]:
            return '3'
        elif x>= quantile_values.loc[0.50,attribute] and x <= quantile_values.loc[0.75, attribute]:
            return '2'
        else:
            return '1'
    else:
        if x <= quantile_values.loc[0.25,attribute]:
            return '1'
        elif x >= quantile_values.loc[0.25,attribute] and x<= quantile_values.loc[0.50,attribute]:
            return '2'
        elif x >= quantile_values.loc[0.50,attribute] and x <= quantile_values.loc[0.75,attribute]:
            return '3'
        else:
            return '4'
    

In [ ]:
rfm_table['r_class'] = rfm_table['recency'].apply(quantile_classes,args = (quantile_df,'recency'))
rfm_table['f_class'] = rfm_table['frequency'].apply(quantile_classes,args = (quantile_df,'frequency'))
rfm_table['m_class'] = rfm_table['monetary'].apply(quantile_classes, args = (quantile_df,'monetary'))

In [ ]:
rfm_table

adding 3 classes into one ''

In [ ]:
rfm_table['total_rfm_score'] = rfm_table[['r_class','f_class']].sum(axis=1)
rfm_table.head(5)

In [ ]:
plt.figure(figsize= (9,6))
rfm_table['r_class'].value_counts(normalize = True).plot(kind='bar')

Some explortary anslysis below using more dimensions


In [ ]:
#reload the dataset
df2 = pd.read_csv('../input/sample-sales-data/sales_data_sample.csv', encoding = 'unicode_escape')

In [ ]:
df2.shape

In [ ]:
df2.columns

In [ ]:
df2 = pd.DataFrame(df2)

In [ ]:
#Removing the variables which dont add significant value fot the analysis.
to_drop = ['ADDRESSLINE1','ADDRESSLINE2','STATE','POSTALCODE']
df2 = df2.drop(to_drop, axis=1)
df2

In [ ]:
plt.figure(figsize = (9,6))
sns.distplot(df2['QUANTITYORDERED'])
plt.title('Quantity Orders Distribution')
plt.xlabel('Quantity Ordered')
plt.ylabel('Frequency')
plt.show()

Majority of the order's quantity are between 20 -40 units.


In [ ]:
plt.figure(figsize = (9,6))
sns.distplot(df2['PRICEEACH'])
plt.title('Price Distribution')
plt.xlabel('Price Ordered')
plt.ylabel('frequency')
plt.show()

the max price per order is $100 and this price range had a high # of orders


In [ ]:
plt.figure(figsize = (9,6))
sns.distplot(df2['SALES'])
plt.xlabel('Sale Distribution')
plt.ylabel('Frequency')
plt.show()

Explore the STATUS of the products/delivery


In [ ]:
df2['STATUS'].value_counts(normalize = True)
# % of xx status of all

check the time frame for the dataset


In [ ]:
df2.groupby(['YEAR_ID'])['MONTH_ID'].nunique()

2005 has partial data for 5 months

In [ ]:
DEALSIZE 


In [ ]:
df2['DEALSIZE'].value_counts()

In [ ]:
plt.figure(figsize = (9,6))
df2['DEALSIZE'].value_counts(normalize = True).plot(kind='bar')
plt.title('deal size distribution')
plt.xlabel('deal size types')
plt.ylabel('% of proportion')
plt.show()

In [ ]:
plt.figure(figsize = (9,6))
df2.groupby(['YEAR_ID'])['SALES'].sum().plot(kind = 'line')
plt.title('Annual Revenue')
plt.xlabel('Year')
plt.ylabel('Revenue')
plt.xticks(np.arange(2003,2006,1))
plt.show()

In [ ]:
plt.figure(figsize = (9,6))
monthly = df2.groupby(['YEAR_ID','MONTH_ID'])['SALES'].sum().reset_index()
monthly
sns.lineplot(x='MONTH_ID',y='SALES', hue= 'YEAR_ID',data = monthly)
plt.xlabel('By Month')
plt.ylabel('monthly revenue')
plt.show()


In [ ]:
MONTHLY Revenue Growth

In [ ]:
monthly['MONTHLY GROWTH'] = monthly['SALES'].pct_change()
monthly.head(5)

In [ ]:
plt.figure(figsize = (9,6))
sns.lineplot(x= 'MONTH_ID',y= 'MONTHLY GROWTH', hue = 'YEAR_ID', data = monthly)
plt.title('Sales Growth Rate by Month')
plt.xlabel('month')
plt.ylabel('sales')
plt.show()

# Average Sales Per Order

In [ ]:
average_sales = df2.groupby(['YEAR_ID','MONTH_ID'])['SALES'].mean().reset_index()
average_sales
plt.figure(figsize = (9,6))
sns.lineplot(x='MONTH_ID', y= 'SALES', hue='YEAR_ID',data = average_sales)
plt.show()

In [ ]:
from pyecharts import Bar